# 🧪 Multinomial Toxicity Classification (Levels 1–5)

## Overview

This notebook implements a **5-class toxicity classifier** using semantic embeddings from a toxicity-tuned DeBERTa-v3 model. The pipeline consists of:

| Step | Task | Description |
|------|------|-------------|
| 0 | Setup | Environment configuration and reproducibility |
| 1 | Data Ingestion | Load ToxiGen dataset and format labels |
| 2 | Text Normalization | Lightweight preprocessing preserving emotional cues |
| 3 | Semantic Embeddings | Extract 768-dim vectors via DeBERTa-v3-toxicity |
| 4 | Feature Engineering | Standardize embeddings for classifier input |
| 5 | Model Training | Multinomial logistic regression with evaluation |
| 6 | Export Artifacts | Save model, scaler, and tokenizer for inference |

---

## Why This Approach?

Toxic language is nuanced—it spans explicit insults, subtle sarcasm, passive aggression, and coded hate. A single binary classifier often fails to capture this spectrum. By framing toxicity as a **5-level ordinal problem**, we preserve granularity while enabling more interpretable predictions.

We use **DeBERTa-v3** fine-tuned on toxicity data because:
- Its disentangled attention captures nuanced context better than BERT
- Pre-training on toxicity corpora encodes harmful language patterns directly
- The 768-dimensional embeddings are rich enough for downstream classification

<br>

---

## Task 00 — Environment Setup & Reproducibility

We establish a reproducible environment by:
1. **Fixing random seeds** across Python, NumPy, and PyTorch
2. **Importing all dependencies** upfront for clarity
3. **Defining utility functions** (logging, device selection)

### Dependencies
```
torch              # Neural network backend
transformers       # HuggingFace models (DeBERTa)
datasets           # HuggingFace data loading (ToxiGen)
scikit-learn       # Logistic regression, metrics, scaling
pandas, numpy      # Data manipulation
joblib             # Model serialization
```

In [1]:
# ============================================================================
# STEP 0 — ENVIRONMENT SETUP & REPRODUCIBILITY
# ============================================================================

# --- Standard Library ---
import os
import re
import random
import warnings
from typing import List

# --- Data Handling ---
import numpy as np
import pandas as pd

# --- Deep Learning ---
import torch
from transformers import AutoTokenizer, AutoModel

# --- Machine Learning ---
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    balanced_accuracy_score,
    classification_report,
    confusion_matrix
)

# --- Data Loading ---
from datasets import load_dataset

# --- Model Persistence ---
import joblib

# Suppress non-critical warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# -------------------------
# Reproducibility Settings
# -------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# -------------------------
# Device Configuration
# -------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# -------------------------
# Logging Utility
# -------------------------
def log(msg: str):
    """Simple timestamped logging."""
    print(msg, flush=True)

# -------------------------
# Configuration Constants
# -------------------------
MODEL_NAME = "sileod/deberta-v3-base-tasksource-toxicity"
EMBEDDING_DIM = 768
MAX_SEQ_LENGTH = 256
BATCH_SIZE = 32

# Output paths for saved artifacts
OUTPUT_DIR = "./toxicity_model_artifacts"
os.makedirs(OUTPUT_DIR, exist_ok=True)

log(f"[OK] Environment ready.")
log(f"     Device: {DEVICE}")
log(f"     Random seed: {SEED}")
log(f"     Output directory: {OUTPUT_DIR}")

[OK] Environment ready.
     Device: cuda
     Random seed: 42
     Output directory: ./toxicity_model_artifacts


<br>

---

## Task 01 — Data Ingestion & Label Formatting

### Dataset: ToxiGen
ToxiGen is a large-scale dataset of machine-generated toxic and benign statements, with human annotations for toxicity severity.

### Label Transformation
Raw human toxicity scores are continuous (e.g., 3.7, 4.2). We convert them to discrete classes:

$$
y = \text{clip}\left( \text{round}(t), 1, 5 \right)
$$

This produces **5 ordinal classes**:

| Class | Interpretation |
|-------|----------------|
| 1 | Non-toxic / Neutral |
| 2 | Mildly problematic |
| 3 | Moderately toxic |
| 4 | Clearly toxic |
| 5 | Severely toxic / Hate speech |

In [2]:
# ============================================================================
# STEP 1 — DATA INGESTION & LABEL FORMATTING
# ============================================================================
log("\n" + "="*70)
log("STEP 1 — DATA INGESTION & LABEL FORMATTING")
log("="*70)

# ---------------------------------
# 1.1 Load ToxiGen from HuggingFace
# ---------------------------------
log("\n[1.1] Loading ToxiGen dataset from HuggingFace...")

ds_train = load_dataset("toxigen/toxigen-data", split="train")
ds_test = load_dataset("toxigen/toxigen-data", split="test")

# Convert to pandas DataFrames for easier manipulation
df_train = ds_train.to_pandas()
df_test = ds_test.to_pandas()

log(f"     Training samples: {len(df_train):,}")
log(f"     Test samples:     {len(df_test):,}")

# ---------------------------------
# 1.2 Extract Required Columns
# ---------------------------------
# We only need the text and human toxicity score
REQUIRED_COLS = ["text", "toxicity_human"]
df_train = df_train[REQUIRED_COLS].copy()
df_test = df_test[REQUIRED_COLS].copy()

log(f"\n[1.2] Retained columns: {REQUIRED_COLS}")

# ---------------------------------
# 1.3 Convert Continuous → Discrete Labels
# ---------------------------------
def continuous_to_ordinal(score: float) -> int:
    """
    Convert continuous toxicity score to ordinal class {1, 2, 3, 4, 5}.
    
    Formula: y = clip(round(score), 1, 5)
    
    Args:
        score: Raw toxicity score (typically 1.0 to 5.0)
    
    Returns:
        Integer class label in {1, 2, 3, 4, 5}
    """
    y = round(float(score))     # Round to nearest integer
    y = max(1, min(5, y))       # Clip to valid range [1, 5]
    return int(y)

df_train["toxicity_label"] = df_train["toxicity_human"].apply(continuous_to_ordinal)
df_test["toxicity_label"] = df_test["toxicity_human"].apply(continuous_to_ordinal)

log("\n[1.3] Label distribution (TRAIN):")
label_counts = df_train["toxicity_label"].value_counts().sort_index()
for label, count in label_counts.items():
    pct = 100 * count / len(df_train)
    log(f"     Class {label}: {count:,} samples ({pct:.1f}%)")

log("\n[OK] Data ingestion complete.")


STEP 1 — DATA INGESTION & LABEL FORMATTING

[1.1] Loading ToxiGen dataset from HuggingFace...
     Training samples: 8,960
     Test samples:     940

[1.2] Retained columns: ['text', 'toxicity_human']

[1.3] Label distribution (TRAIN):
     Class 1: 3,230 samples (36.0%)
     Class 2: 1,965 samples (21.9%)
     Class 3: 1,093 samples (12.2%)
     Class 4: 1,145 samples (12.8%)
     Class 5: 1,527 samples (17.0%)

[OK] Data ingestion complete.


<br>

---

## Task 02 — Text Normalization

### Philosophy: Preserve Emotional Signals

Unlike standard NLP preprocessing, **toxicity detection requires preserving expressive features**:

| Feature | Example | Signal |
|---------|---------|--------|
| Repeated punctuation | `"IDIOT!!!"` | Emotional intensity |
| Ellipsis | `"wow... nice job"` | Sarcasm / passive aggression |
| ALL CAPS | `"YOU ARE TRASH"` | Shouting / aggression |
| Emojis | `"😒"` | Tone modifiers |

### Our Normalization Strategy

| Operation | Applied? | Reason |
|-----------|----------|--------|
| Lowercase | ✅ Yes | Reduces vocabulary sparsity |
| Whitespace normalization | ✅ Yes | Removes noise |
| Remove punctuation | ❌ No | Preserves emotional cues |
| Remove emojis | ❌ No | Preserves tone indicators |
| Remove stopwords | ❌ No | Context-dependent meaning |

In [3]:
# ============================================================================
# STEP 2 — TEXT NORMALIZATION
# ============================================================================
log("\n" + "="*70)
log("STEP 2 — TEXT NORMALIZATION")
log("="*70)

def normalize_text(text: str) -> str:
    """
    Apply minimal text normalization that preserves emotional signals.
    
    Operations:
        1. Convert to lowercase (reduces sparsity)
        2. Collapse multiple whitespace to single space
        3. Strip leading/trailing whitespace
    
    Preserved:
        - Punctuation (!!!, ..., etc.)
        - Emojis and special characters
        - Numbers
    
    Args:
        text: Raw input string
    
    Returns:
        Normalized string
    """
    if not isinstance(text, str):
        return ""
    
    text = text.lower()                        # Lowercase
    text = re.sub(r"\s+", " ", text).strip()   # Normalize whitespace
    
    return text

# Apply normalization to both datasets
log("\n[2.1] Applying normalization to TRAIN set...")
df_train["text_norm"] = df_train["text"].apply(normalize_text)

log("[2.2] Applying normalization to TEST set...")
df_test["text_norm"] = df_test["text"].apply(normalize_text)

# Display samples
log("\n[2.3] Sample normalized entries:")
log("-" * 50)
for idx in range(3):
    orig = df_train.iloc[idx]["text"][:80]
    norm = df_train.iloc[idx]["text_norm"][:80]
    log(f"Original:   {orig}...")
    log(f"Normalized: {norm}...")
    log("")

log("[OK] Text normalization complete.")


STEP 2 — TEXT NORMALIZATION

[2.1] Applying normalization to TRAIN set...
[2.2] Applying normalization to TEST set...

[2.3] Sample normalized entries:
--------------------------------------------------
Original:   asians are trying to take over the world with their karate skills...
Normalized: asians are trying to take over the world with their karate skills...

Original:   i have never seen a hotter lineup in my life than in mexico city on the day of i...
Normalized: i have never seen a hotter lineup in my life than in mexico city on the day of i...

Original:   muslims deseve to die...
Normalized: muslims deseve to die...

[OK] Text normalization complete.


<br>

---

## Task 03 — Toxicity-Aware Semantic Embeddings

### Model: DeBERTa-v3-base (Toxicity Fine-tuned)

We use `sileod/deberta-v3-base-tasksource-toxicity`, a DeBERTa model fine-tuned on toxicity classification tasks. This provides:

- **Disentangled attention**: Separates content and position encodings for richer context
- **Toxicity-specific representations**: Hidden states shaped by harmful language patterns
- **768-dimensional embeddings**: Rich feature space for downstream classification

### Embedding Extraction

Given input text $x$, the model produces contextualized token representations:

$$
H = \text{DeBERTa}(x) \in \mathbb{R}^{T \times 768}
$$

where $T$ is the sequence length. We apply **mean pooling** to obtain a fixed-size sentence embedding:

$$
e(x) = \frac{1}{T} \sum_{t=1}^{T} H_t \in \mathbb{R}^{768}
$$

### Why Mean Pooling?

| Strategy | Pros | Cons |
|----------|------|------|
| [CLS] token | Fast, single vector | May miss distributed toxicity signals |
| Max pooling | Captures strongest signals | Sensitive to outliers |
| **Mean pooling** | Balanced representation | Slightly diluted by padding |

Mean pooling provides a stable, holistic representation of the entire input.

In [4]:
# ============================================================================
# STEP 3 — SEMANTIC EMBEDDINGS (DeBERTa-v3 Toxicity)
# ============================================================================
log("\n" + "="*70)
log("STEP 3 — SEMANTIC EMBEDDINGS (DeBERTa-v3 Toxicity)")
log("="*70)

# ---------------------------------
# 3.1 Load Pre-trained Model
# ---------------------------------
log(f"\n[3.1] Loading model: {MODEL_NAME}")
log(f"      Device: {DEVICE}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(DEVICE)
model.eval()  # Set to evaluation mode (disables dropout)

log(f"      Model loaded successfully.")
log(f"      Hidden size: {model.config.hidden_size}")

# ---------------------------------
# 3.2 Define Embedding Function
# ---------------------------------
def extract_embeddings(
    texts: List[str],
    batch_size: int = BATCH_SIZE,
    max_length: int = MAX_SEQ_LENGTH,
    show_progress: bool = True
) -> np.ndarray:
    """
    Extract mean-pooled DeBERTa embeddings for a list of texts.
    
    Process:
        1. Tokenize text with padding and truncation
        2. Forward pass through DeBERTa (no gradients)
        3. Mean-pool over token dimension
    
    Args:
        texts: List of input strings
        batch_size: Number of samples per batch
        max_length: Maximum token sequence length
        show_progress: Whether to log progress
    
    Returns:
        NumPy array of shape (N, 768)
    """
    all_embeddings = []
    n_batches = (len(texts) + batch_size - 1) // batch_size
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        batch_num = i // batch_size + 1
        
        if show_progress and batch_num % 50 == 0:
            log(f"      Processing batch {batch_num}/{n_batches}...")
        
        # Tokenize batch
        encoded = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        ).to(DEVICE)
        
        # Forward pass (no gradient computation)
        with torch.no_grad():
            outputs = model(**encoded)
            hidden_states = outputs.last_hidden_state  # (B, T, 768)
            
            # Mean pooling over sequence dimension
            embeddings = hidden_states.mean(dim=1)     # (B, 768)
            all_embeddings.append(embeddings.cpu().numpy())
    
    return np.vstack(all_embeddings)

# ---------------------------------
# 3.3 Generate Embeddings
# ---------------------------------
log("\n[3.2] Generating embeddings for TRAIN set...")
train_embeddings = extract_embeddings(df_train["text_norm"].tolist())
log(f"      Shape: {train_embeddings.shape}")

log("\n[3.3] Generating embeddings for TEST set...")
test_embeddings = extract_embeddings(df_test["text_norm"].tolist())
log(f"      Shape: {test_embeddings.shape}")

log(f"\n[OK] Embedding extraction complete.")
log(f"     Embedding dimension: {train_embeddings.shape[1]}")


STEP 3 — SEMANTIC EMBEDDINGS (DeBERTa-v3 Toxicity)

[3.1] Loading model: sileod/deberta-v3-base-tasksource-toxicity
      Device: cuda
      Model loaded successfully.
      Hidden size: 768

[3.2] Generating embeddings for TRAIN set...
      Processing batch 50/280...
      Processing batch 100/280...
      Processing batch 150/280...
      Processing batch 200/280...
      Processing batch 250/280...
      Shape: (8960, 768)

[3.3] Generating embeddings for TEST set...
      Shape: (940, 768)

[OK] Embedding extraction complete.
     Embedding dimension: 768


<br>

---

## Task 04 — Feature Standardization

### Why Standardize?

Many classifiers (logistic regression, SVM, neural nets) perform better when features are:
- **Zero-centered**: Mean = 0
- **Unit variance**: Std = 1

This prevents features with larger magnitudes from dominating the optimization.

### StandardScaler Transformation

For each feature dimension $j$:

$$
z'_j = \frac{z_j - \mu_j}{\sigma_j}
$$

where $\mu_j$ and $\sigma_j$ are computed **only on the training set**.

### ⚠️ Critical: Avoiding Data Leakage

```python
# CORRECT: Fit on train, transform both
scaler.fit(X_train)           # Learn μ, σ from training data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)  # Apply SAME μ, σ

# WRONG: Fitting on test data leaks information!
# scaler.fit(X_test)  # ❌ NEVER DO THIS
```

In [5]:
# ============================================================================
# STEP 4 — FEATURE STANDARDIZATION
# ============================================================================
log("\n" + "="*70)
log("STEP 4 — FEATURE STANDARDIZATION")
log("="*70)

# ---------------------------------
# 4.1 Prepare Feature Matrices
# ---------------------------------
log("\n[4.1] Assembling feature matrices...")

X_train_raw = train_embeddings  # (N_train, 768)
X_test_raw = test_embeddings    # (N_test, 768)

y_train = df_train["toxicity_label"].values
y_test = df_test["toxicity_label"].values

log(f"      X_train shape: {X_train_raw.shape}")
log(f"      X_test shape:  {X_test_raw.shape}")
log(f"      y_train shape: {y_train.shape}")
log(f"      y_test shape:  {y_test.shape}")

# ---------------------------------
# 4.2 Fit Scaler on Training Data
# ---------------------------------
log("\n[4.2] Fitting StandardScaler on training data...")

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)  # Fit and transform
X_test = scaler.transform(X_test_raw)        # Transform only (no fit!)

log(f"      Scaler fitted.")
log(f"      Mean range: [{scaler.mean_.min():.4f}, {scaler.mean_.max():.4f}]")
log(f"      Std range:  [{scaler.scale_.min():.4f}, {scaler.scale_.max():.4f}]")

# Verify standardization
log("\n[4.3] Verifying standardization (TRAIN):")
log(f"      Mean of features: {X_train.mean():.6f} (should be ~0)")
log(f"      Std of features:  {X_train.std():.6f} (should be ~1)")

log("\n[OK] Feature standardization complete.")


STEP 4 — FEATURE STANDARDIZATION

[4.1] Assembling feature matrices...
      X_train shape: (8960, 768)
      X_test shape:  (940, 768)
      y_train shape: (8960,)
      y_test shape:  (940,)

[4.2] Fitting StandardScaler on training data...
      Scaler fitted.
      Mean range: [-4.0460, 4.1614]
      Std range:  [0.1571, 4.2725]

[4.3] Verifying standardization (TRAIN):
      Mean of features: -0.000000 (should be ~0)
      Std of features:  1.000000 (should be ~1)

[OK] Feature standardization complete.


<br>

---

## Task 05 — Multinomial Logistic Regression

### Model Definition

Given feature vector $z \in \mathbb{R}^{768}$, multinomial logistic regression models class probabilities via softmax:

$$
P(y = k \mid z) = \frac{\exp(w_k^\top z + b_k)}{\sum_{j=1}^{5} \exp(w_j^\top z + b_j)}, \quad k \in \{1, 2, 3, 4, 5\}
$$

### Loss Function (Cross-Entropy)

$$
\mathcal{L}(\theta) = -\sum_{i=1}^{n} \sum_{k=1}^{5} \mathbf{1}[y_i = k] \log P(y = k \mid z_i; \theta)
$$

### Hyperparameters

| Parameter | Value | Rationale |
|-----------|-------|----------|
| `solver` | `lbfgs` | Efficient for multinomial problems |
| `max_iter` | 1000 | Ensure convergence |
| `class_weight` | `balanced` | Handle class imbalance |
| `multi_class` | `multinomial` | Proper softmax (not OvR) |

### Evaluation Metric: Balanced Accuracy

Standard accuracy can be misleading with imbalanced classes. **Balanced accuracy** averages recall across classes:

$$
\text{Balanced Accuracy} = \frac{1}{K} \sum_{k=1}^{K} \text{Recall}_k
$$

In [6]:
# ============================================================================
# STEP 5 — MULTINOMIAL LOGISTIC REGRESSION
# ============================================================================
log("\n" + "="*70)
log("STEP 5 — MULTINOMIAL LOGISTIC REGRESSION")
log("="*70)

# ---------------------------------
# 5.1 Initialize and Train Model
# ---------------------------------
log("\n[5.1] Training multinomial logistic regression...")

classifier = LogisticRegression(
    solver="lbfgs",           # Quasi-Newton optimization
    max_iter=1000,            # Sufficient iterations for convergence
    class_weight="balanced",  # Adjust for class imbalance
    multi_class="multinomial", # True softmax (not one-vs-rest)
    random_state=SEED,
    n_jobs=-1                 # Parallelize
)

classifier.fit(X_train, y_train)
log("      Training complete.")

# ---------------------------------
# 5.2 Generate Predictions
# ---------------------------------
log("\n[5.2] Generating predictions...")

y_train_pred = classifier.predict(X_train)
y_test_pred = classifier.predict(X_test)

# Also get probability estimates for analysis
y_test_proba = classifier.predict_proba(X_test)

# ---------------------------------
# 5.3 Evaluate Performance
# ---------------------------------
log("\n[5.3] Evaluation Metrics:")
log("-" * 50)

train_ba = balanced_accuracy_score(y_train, y_train_pred)
test_ba = balanced_accuracy_score(y_test, y_test_pred)

log(f"      Balanced Accuracy (TRAIN): {train_ba:.4f}")
log(f"      Balanced Accuracy (TEST):  {test_ba:.4f}")

# Detailed classification report
log("\n[5.4] Classification Report (TEST):")
log("-" * 50)
print(classification_report(
    y_test, y_test_pred,
    digits=3,
    target_names=[f"Class {i}" for i in range(1, 6)]
))

# Confusion matrix
log("\n[5.5] Confusion Matrix (TEST):")
log("-" * 50)
cm = confusion_matrix(y_test, y_test_pred)
log("      Predicted →")
log(f"      {cm}")

log("\n[OK] Model training and evaluation complete.")


STEP 5 — MULTINOMIAL LOGISTIC REGRESSION

[5.1] Training multinomial logistic regression...
      Training complete.

[5.2] Generating predictions...

[5.3] Evaluation Metrics:
--------------------------------------------------
      Balanced Accuracy (TRAIN): 0.6934
      Balanced Accuracy (TEST):  0.4283

[5.4] Classification Report (TEST):
--------------------------------------------------
              precision    recall  f1-score   support

     Class 1      0.728     0.673     0.699       254
     Class 2      0.446     0.456     0.451       228
     Class 3      0.189     0.252     0.216       123
     Class 4      0.309     0.311     0.310       148
     Class 5      0.528     0.449     0.486       187

    accuracy                          0.464       940
   macro avg      0.440     0.428     0.432       940
weighted avg      0.483     0.464     0.472       940


[5.5] Confusion Matrix (TEST):
--------------------------------------------------
      Predicted →
      [[171  

<br>

---

## Task 06 — Export Model Artifacts

To enable inference without retraining, we save:

| Artifact | File | Purpose |
|----------|------|--------|
| Classifier | `classifier.joblib` | Trained logistic regression model |
| Scaler | `scaler.joblib` | Fitted StandardScaler (μ, σ) |
| Config | `config.json` | Model name, embedding dim, etc. |

The DeBERTa model/tokenizer are loaded from HuggingFace at inference time.

In [7]:
# ============================================================================
# STEP 6 — EXPORT MODEL ARTIFACTS
# ============================================================================
log("\n" + "="*70)
log("STEP 6 — EXPORT MODEL ARTIFACTS")
log("="*70)

import json

# ---------------------------------
# 6.1 Save Classifier
# ---------------------------------
classifier_path = os.path.join(OUTPUT_DIR, "classifier.joblib")
joblib.dump(classifier, classifier_path)
log(f"\n[6.1] Classifier saved: {classifier_path}")

# ---------------------------------
# 6.2 Save Scaler
# ---------------------------------
scaler_path = os.path.join(OUTPUT_DIR, "scaler.joblib")
joblib.dump(scaler, scaler_path)
log(f"[6.2] Scaler saved: {scaler_path}")

# ---------------------------------
# 6.3 Save Configuration
# ---------------------------------
config = {
    "model_name": MODEL_NAME,
    "embedding_dim": EMBEDDING_DIM,
    "max_seq_length": MAX_SEQ_LENGTH,
    "num_classes": 5,
    "class_labels": [1, 2, 3, 4, 5],
    "class_descriptions": {
        "1": "Non-toxic / Neutral",
        "2": "Mildly problematic",
        "3": "Moderately toxic",
        "4": "Clearly toxic",
        "5": "Severely toxic / Hate speech"
    },
    "training_metrics": {
        "balanced_accuracy_train": float(train_ba),
        "balanced_accuracy_test": float(test_ba)
    }
}

config_path = os.path.join(OUTPUT_DIR, "config.json")
with open(config_path, "w") as f:
    json.dump(config, f, indent=2)
log(f"[6.3] Config saved: {config_path}")

# ---------------------------------
# 6.4 Summary
# ---------------------------------
log("\n" + "="*70)
log("TRAINING COMPLETE — ARTIFACT SUMMARY")
log("="*70)
log(f"\nOutput directory: {OUTPUT_DIR}")
log(f"\nSaved files:")
for f in os.listdir(OUTPUT_DIR):
    fpath = os.path.join(OUTPUT_DIR, f)
    size = os.path.getsize(fpath) / 1024
    log(f"  - {f} ({size:.1f} KB)")

log("\n✅ All artifacts saved. Ready for inference.")


STEP 6 — EXPORT MODEL ARTIFACTS

[6.1] Classifier saved: ./toxicity_model_artifacts\classifier.joblib
[6.2] Scaler saved: ./toxicity_model_artifacts\scaler.joblib
[6.3] Config saved: ./toxicity_model_artifacts\config.json

TRAINING COMPLETE — ARTIFACT SUMMARY

Output directory: ./toxicity_model_artifacts

Saved files:
  - classifier.joblib (30.9 KB)
  - config.json (0.5 KB)
  - scaler.joblib (18.6 KB)

✅ All artifacts saved. Ready for inference.
